In [1]:
import pandas as pd

In [2]:
dataset = pd.read_parquet("../data/cleaned/dataset_clean.parquet")

In [3]:
# Define the helper function
def assign_feature(value, condition_func):
    """
    Assigns 1 or 0 based on the condition_func.
    If the value is None (NaN), returns None.
    """
    if pd.isna(value):
        return None
    return 1 if condition_func(value) else 0

dataset['date_accident'] =  pd.to_datetime(dataset['jour_accident'].astype(str) + '/' + dataset['mois_accident'].astype(str) + '/' + dataset['annee_accident'].astype(str), dayfirst=True)
dataset['age'] = dataset.apply(
    lambda row: row['date_accident'].year - row['annee_naissance'] 
    if (pd.notna(row['annee_naissance']))&(pd.notna(row['date_accident'])) 
    else None, axis=1
).astype('Int64')

dataset['accident_pendant_la_journee'] = dataset['heure_accident'].apply(lambda heure: 1 if (6 <= int(heure[:2]) < 18) else 0)

dataset['limite_de_vitesse_elevee'] = dataset['vitesse_max_autorisee'].apply(lambda vma: assign_feature(vma, lambda x: x > 80))
dataset['limite_de_vitesse_faible'] = dataset['vitesse_max_autorisee'].apply(lambda vma: assign_feature(vma, lambda x: x < 50 ))
dataset['est_en_agglomeration'] = dataset['localisation'].apply(lambda agg: assign_feature(agg, lambda x: x == 2))
dataset['est_une_route_principale'] = dataset['categorie_route'].apply(lambda cr: assign_feature(cr, lambda x: x not in [5, 6, 9]))  # (parking, voies prives, et autres))
dataset['est_une_voie_a_sens_unique'] = dataset['regime_circulation'].apply(lambda sc: assign_feature(sc, lambda x: x == 1))
dataset['est_une_route_etroite'] = dataset['larrout'].apply(lambda lc: assign_feature(lc, lambda x: x < 4))
dataset['est_une_route_condition_normale'] = dataset['etat_surface'].apply(lambda es: assign_feature(es, lambda x: x == 1))

dataset['est_un_vehicule_leger'] = dataset['categorie_vehicule'].apply(lambda cat: assign_feature(cat, lambda x: x in [2, 3, 4, 5, 6, 30, 31, 32, 34, 35, 36, 41, 42, 43]))  # (moto, scooter, etc))
dataset['est_une_voiture_personelle'] = dataset['categorie_vehicule'].apply(lambda cat: assign_feature(cat, lambda x: x == 7))  # (voiture personelle))
dataset['est_un_velo'] = dataset['categorie_vehicule'].apply(lambda cat: assign_feature(cat, lambda x: x == 1))  # (velo))
dataset['est_un_vehicule_lourd'] = dataset['categorie_vehicule'].apply(lambda cat: assign_feature(cat, lambda x: x in [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 37, 38, 39, 40]))  # (camion, bus, etc))

# dataset['est_une_femme'] = dataset['sexe_usager'].apply(lambda sexe: assign_feature(sexe, lambda x: x == 2))
dataset['jeune_conducteur'] = dataset['age'].apply(lambda age: assign_feature(age, lambda x: x <= 25))
dataset['conducteur_vieux'] = dataset['age'].apply(lambda age: assign_feature(age, lambda x: x > 65))
dataset['est_un_conducteur'] = dataset['categorie_usager'].apply(lambda qu: assign_feature(qu, lambda x: x == 1))  # (conducteur))
dataset['est_equipe_securite'] = dataset['equipement_securite_1'].apply(lambda eq: assign_feature(eq, lambda x: x in [1, 2, 3, 4, 5, 6, 7]))  # (airbag etc...))
dataset['est_equipe_securite_2'] = dataset['equipement_securite_2'].apply(lambda eq: assign_feature(eq, lambda x: x in [1, 2, 3, 4, 5, 6, 7]))
dataset['est_un_vehicule_electrique'] = dataset['type_motorisation'].apply(lambda tm: assign_feature(tm, lambda x: x in [2, 3]))  # (electrique et hybride))
dataset['est_un_vehicule_thermique'] = dataset['type_motorisation'].apply(lambda tm: assign_feature(tm, lambda x: x == 1))  # (essence et diesel))

dataset['conditions_lumineuses_defavorables'] = dataset['conditions_lumieres'].apply(lambda cl: assign_feature(cl, lambda x: x != 1))  # (pendant la journée))
dataset['conditions_meteorologiques_defavorables'] = dataset['conditions_atmospheriques'].apply(lambda cm: assign_feature(cm, lambda x: x not in [1, 2, 8]))  # (normal, pluie legere, temps couvert))
dataset['a_hurte_un_obstacle_fixe'] = dataset['obstacle_fixe_heurte'].apply(lambda of: assign_feature(of, lambda x: x not in [0, -1]))
dataset['a_hurte_un_obstacle_mobile'] = dataset['obstacle_mobile_heurte'].apply(lambda of: assign_feature(of, lambda x: x not in [0, -1]))

def gestion_accident(blessure):
    if pd.isna(blessure):
        return None
    elif blessure == 1:
        return 0  # rien
    elif blessure == 2:
        return 10  # tué
    elif blessure == 3:
        return 5  # hospitalisé
    elif blessure == 4:
        return 1  # blessé léger
    else:
        return None  # For any other unexpected values

dataset['cout_accident'] = dataset['gravite_blessure'].apply(gestion_accident)

In [4]:
colones_a_garder = [
    #identifiants
    'identifiant_vehicule', 'numero_vehicule', 'numero_accident', 'identifiant_usager',
    #usager
    'conducteur_vieux', 'jeune_conducteur', 'est_une_femme', 'est_equipe_securite','est_equipe_securite_2','est_un_conducteur',
    #lieux
    'limite_de_vitesse_elevee','limite_de_vitesse_faible', 'est_une_route_principale' ,'est_en_agglomeration',
    'conditions_lumineuses_defavorables','conditions_meteorologiques_defavorables', 'accident_pendant_la_journee',
    'a_hurte_un_obstacle_fixe', 'a_hurte_un_obstacle_mobile', 'est_une_voie_a_sens_unique', 'est_une_route_etroite',
    'est_une_route_condition_normale',
    #vehicule
    'est_un_vehicule_leger','est_une_voiture_personelle', 'est_un_velo','est_un_vehicule_lourd', 
    'est_un_vehicule_thermique', 'est_un_vehicule_electrique', 
    #target
    'cout_accident'
]


colonnes_a_supprimer = [
    #date de l'accident
    'jour_accident', 'mois_accident', 'annee_accident', 'heure_accident','date_accident',
     
    #information converties en feature
    'age', 'conditions_lumieres', 'localisation', 'vitesse_max_autorisee',
    'conditions_atmospheriques', 'categorie_route', 'regime_circulation', 
    'sexe_usager', 'annee_naissance', 'obstacle_fixe_heurte', 'obstacle_mobile_heurte',
    'categorie_vehicule', 'type_motorisation', 'categorie_usager', 'equipement_securite_1',
    'equipement_securite_2', 'larrout', 'etat_surface','gravite_blessure',

    #a traiter?
    'nombre_voies', 'presence_voie_reservee', 'profil_route', 
    'numero_pr','distance_pr', 'tracage_route',  
    'infrastructure', 'situation_accident',
    'point_choc_initial', 'manoeuvre_avant_accident',
    'place_occupant', 'motif_deplacement',
    'localisation_pieton', 'action_pieton', 'etat_pieton'
    'departement', 'type_intersection',

    #a jeter
    'sens_circulation','commune', 'equipement_securite_3', 'type_collision',
]

dataset = dataset[colones_a_garder]

In [5]:
#il faut creer un jeu de donnes qui parle des clients de l'assurance => les conducteurs.
#il faut un dataset qui aggrege les degats causés par un conducteur dans son accident.

total_cost = dataset.groupby('numero_accident')['cout_accident'].sum().reset_index()
total_cost = total_cost.rename(columns={'cout_accident': 'total_cout_accident'})

drivers = dataset[dataset['est_un_conducteur'] == 1].copy()

agg_data = drivers.merge(total_cost, on='numero_accident', how='left')

In [6]:
agg_data.columns

Index(['identifiant_vehicule', 'numero_vehicule', 'numero_accident',
       'identifiant_usager', 'conducteur_vieux', 'jeune_conducteur',
       'est_une_femme', 'est_equipe_securite', 'est_equipe_securite_2',
       'est_un_conducteur', 'limite_de_vitesse_elevee',
       'limite_de_vitesse_faible', 'est_une_route_principale',
       'est_en_agglomeration', 'conditions_lumineuses_defavorables',
       'conditions_meteorologiques_defavorables',
       'accident_pendant_la_journee', 'a_hurte_un_obstacle_fixe',
       'a_hurte_un_obstacle_mobile', 'est_une_voie_a_sens_unique',
       'est_une_route_etroite', 'est_une_route_condition_normale',
       'est_un_vehicule_leger', 'est_une_voiture_personelle', 'est_un_velo',
       'est_un_vehicule_lourd', 'est_un_vehicule_thermique',
       'est_un_vehicule_electrique', 'cout_accident', 'total_cout_accident'],
      dtype='object')

In [7]:
colonnes_a_sup = ['identifiant_vehicule', 'numero_vehicule','identifiant_usager', 'numero_accident']
agg_data = agg_data.drop(columns=colonnes_a_sup)
agg_data = agg_data.astype({col: 'category' for col in agg_data.columns if col not in ['cout_accident', 'total_cout_accident']})


In [8]:
agg_data.shape

(278935, 26)

In [9]:
agg_data.to_parquet('../data/dataset_features.parquet') #pickle